In [1]:
import math
import random as rd
import matplotlib.pyplot as plt
import cvxpy as cp
import numpy as np

In [2]:
# 파라미터 설정
T = 24  # 시간
S = 100  # 몬테카를로 시뮬레이션 샘플 수

In [3]:
# 결정 변수
x = {t: cp.Variable(nonneg=True) for t in range(T)} # 서버 on 수
x_H = {t: cp.Variable(nonneg=True) for t in range(T)}  # 고화질 서버 수
x_L = {t: cp.Variable(nonneg=True) for t in range(T)}  # 저화질 서버 수
w = {t: cp.Variable(nonneg=True) for t in range(T)}  # 서버 변화량
w_H = {t: cp.Variable(nonneg=True) for t in range(T)}  # 고화질 서버 변화량
w_L = {t: cp.Variable(nonneg=True) for t in range(T)}  # 저화질 서버 변화량
a_H = {(t,s): cp.Variable(nonneg=True) for t in range(T) for s in range(S)}  # 고화질 요청 수
a_L = {(t,s): cp.Variable(nonneg=True) for t in range(T) for s in range(S)}  # 저화질 요청 수
b_H = {(t,s): cp.Variable(nonneg=True) for t in range(T) for s in range(S)}  # 고화질 미충족 요청 수
b_L = {(t,s): cp.Variable(nonneg=True) for t in range(T) for s in range(S)}  # 저화질 미충족 요청 수

In [4]:
# 파라미터 값
P_H = 15  # 고화질 가격
P_L = 10  # 저화질 가격
Z = 1000  # 서버 용량
Z_H = 10  # 고화질 대역폭
Z_L = 2   # 저화질 대역폭
C_H = 1.5 # 고화질 서비스 비용
C_L = 0.5 # 저화질 서비스 비용
C_on = 5  # 서버 에너지 비용
C_m = 2   # 유지보수 비용
F_H = 20  # 고화질 요청 미충족 패널티
F_L = 10  # 저화질 요청 미충족 패널티
pi = 1/S

In [5]:
# alpha, mode determining stage
mean = 374
std = 1021
var = std ** 2

alpha = ((2 * var) + mean ** 2 + (4 * var ** 2 + mean ** 4) ** 0.5) / (2 * var)
min = mean * (alpha - 1) / alpha

# monte carlo

def inverse(p):
  return min / (1 - p) ** (1 / alpha)

Dv = []

for i in range(10000):
  q = inverse(rd.random())
  if q < 1000:
    Dv.append(q)

# plt.figure(figsize=(12,6))
# plt.subplot(121)
# plt.hist(Dv, density=True, bins=100)
# plt.subplot(122)
# plt.hist(Dv, density=True, cumulative=True, bins=100)
# plt.show()

# choose one for each

means = [374, 241, 178, 89, 93, 103, 156, 201, 319, 527, 699, 743, 1458, 1021, 856, 1672, 923, 467, 584, 992, 642, 592, 855, 604]
stds = [1021, 501, 255, 102, 151, 409, 666, 256, 684, 927, 772, 902, 1932, 2193, 1228, 2055, 1327, 1291, 841, 2231, 836, 901, 1127, 1307]
vars = [i ** 2 for i in stds]
alphas = [(((2 * vars[i]) + means[i] ** 2 + (4 * vars[i] ** 2 + means[i] ** 4) ** 0.5) / (2 * vars[i])) for i in range(24)]
mins = [(means[i] * (alphas[i] - 1) / alphas[i]) for i in range(24)]

def inverse_d(p, min, alpha):
  return min / (1 - p) ** (1 / alpha)


In [6]:
rd.seed(1)
D = {}
for i in range(24):
    D[i] = {}  # 각 i에 대해 딕셔너리 초기화
    for s in range(S):
        D[i,s] = inverse_d(rd.random(), mins[i], alphas[i])

In [7]:
rd.seed(2)
alpha_H = {}
alpha_L = {}
for i in range(24):
    alpha_H[i] = {}
    alpha_L[i] = {}  # 각 i에 대해 딕셔너리 초기화
    for s in range(S):
        alpha_H[i,s] = 0.5 + 0.1 * math.cos(i / 12 * math.pi) + rd.random() / 5 - 0.1
        alpha_L[i,s] = 1 - alpha_H[i,s]

In [8]:
# time = [i for i in range(24)]
# plt.plot(time, alpha_H)
# plt.plot(time, alpha_L)
# plt.show()

In [9]:
obj_func = sum(
    - C_on * x[t] - C_m * w[t] for t in range(T)
    ) + sum(
        pi * ((P_H-C_H) * a_H[t,s] + (P_L-C_L) * a_L[t,s] - F_H * b_H[t,s] - F_L * b_L[t,s]) 
        for t in range(T) for s in range(S))

In [10]:
constraints = []
# 서버 수 제약
for t in range(T):
    # constraints.append(x[t] <= N)  # 서버 수
    constraints.append(x_H[t] + x_L[t] == x[t])

# 서버 변화량 제약
for t in range(1, T):
    constraints.append(w_H[t] >= x_H[t] - x_H[t-1])  # 고화질 서버 변화량 제약 1
    constraints.append(w_H[t] >= x_H[t-1] - x_H[t])  # 고화질 서버 변화량 제약 2
    constraints.append(w_L[t] >= x_L[t] - x_L[t-1])  # 저화질 서버 변화량 제약 1
    constraints.append(w_L[t] >= x_L[t-1] - x_L[t])  # 저화질 서버 변화량 제약 2
    constraints.append(w[t] == w_H[t] + w_L[t])  # 서버 변화량의 총합 제약

# 용량 제약
for t in range(T):
    for s in range(S):
        constraints.append(Z_H * a_H[t,s] <= Z * x_H[t]) 
        constraints.append(Z_L * a_L[t,s] <= Z * x_L[t]) 

# 수요 제약
for t in range(T):
    for s in range(S):
        constraints.append(a_H[t,s] + b_H[t,s] == alpha_H[t,s] * D[t,s])  # 고화질 수요
        constraints.append(a_L[t,s] + b_L[t,s] == alpha_L[t,s] * D[t,s])  # 저화질 수요

In [11]:
pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [12]:
prob = cp.Problem(cp.Maximize(obj_func), constraints)
prob.solve(solver='GUROBI')

/opt/anaconda3/lib/python3.12/site-packages/cvxpy/problems/problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


Set parameter Username
Set parameter LicenseID to value 2611964
Academic license - for non-commercial use only - expires 2026-01-20


160153.45937605813

(a) How many servers should be on during each hour of the day?

In [1]:
for t in range(T):
    print(f"Hour {t}: x = {x[t].value}")

NameError: name 'T' is not defined

(b) How should the available bandwidth during an hour be partitioned into high and low-quality videos?

In [55]:
expected_alpha_H = {}
for t in range(T):
    expected_alpha_H[t] = sum(pi * alpha_H[t,s] for s in range(S))

expected_alpha_L = {}
for t in range(T):
    expected_alpha_L[t] = sum(pi * alpha_L[t,s] for s in range(S))

In [56]:
for t in range(T):
    print(f"Hour {t}: alpha_H = {round(expected_alpha_H[t]*100, 2)}, alpha_L = {round(expected_alpha_L[t]*100, 2)}, H% = {round(x_H[t].value/x[t].value*100, 2)}%, L% = {round(x_L[t].value/x[t].value*100, 2)}%")

Hour 0: alpha_H = 59.9, alpha_L = 40.1, H% = 89.34%, L% = 10.66%
Hour 1: alpha_H = 59.87, alpha_L = 40.13, H% = 86.27%, L% = 13.73%
Hour 2: alpha_H = 58.92, alpha_L = 41.08, H% = 88.5%, L% = 11.5%
Hour 3: alpha_H = 56.97, alpha_L = 43.03, H% = 84.37%, L% = 15.63%
Hour 4: alpha_H = 55.2, alpha_L = 44.8, H% = 82.62%, L% = 17.38%
Hour 5: alpha_H = 52.42, alpha_L = 47.58, H% = 83.8%, L% = 16.2%
Hour 6: alpha_H = 50.26, alpha_L = 49.74, H% = 77.97%, L% = 22.03%
Hour 7: alpha_H = 47.88, alpha_L = 52.12, H% = 78.59%, L% = 21.41%
Hour 8: alpha_H = 44.87, alpha_L = 55.13, H% = 81.58%, L% = 18.42%
Hour 9: alpha_H = 42.85, alpha_L = 57.15, H% = 82.62%, L% = 17.38%
Hour 10: alpha_H = 41.3, alpha_L = 58.7, H% = 81.17%, L% = 18.83%
Hour 11: alpha_H = 40.57, alpha_L = 59.43, H% = 75.69%, L% = 24.31%
Hour 12: alpha_H = 39.52, alpha_L = 60.48, H% = 76.43%, L% = 23.57%
Hour 13: alpha_H = 40.27, alpha_L = 59.73, H% = 75.2%, L% = 24.8%
Hour 14: alpha_H = 41.12, alpha_L = 58.88, H% = 79.49%, L% = 20.51%
Ho